**Emotion Recognition: Understanding Group Emotions through Individual Facial Analysis.**

The Emotion Recognition code combines face detection using MTCNN (Multi-Task Cascaded Convolutional Networks) with emotion recognition using a ResNet model to analyze and annotate emotions in images of faces stored in a specified folder using a ResNet, HCB,CBAM and ESA model trained on the FER2013 dataset:

a.) **RESNET**: *epochs=30 (Good Accuracy and prediction).*
b.) **Hierarchical Convolutional Block (HCB)**: *epochs=30 (Good Accuracy and prediction).*
c.) **Convolution Block Attention Module (CBAM)**:*epochs=2 (Bad Accuracy and prediction due to epochs=2).*
d.) **Enhanced Spatial Attention (ESA)**:*epochs=2 (Bad Accuracy and prediction due to epochs=2).*

**Summary:**
1. Load the FER2013 dataset containing facial images and labels.
2. Preprocess the data by extracting images and labels, normalizing pixel values, and encoding the labels.
3. Split the dataset into training and testing sets.
4. Load the pre-trained <ResNet> model.
5. Evaluate the model on the testing set.
6. Print a classification report and confusion matrix to assess the model's performance.
7. Calculate Intersection over Union (IoU), True Positives (TP), False Positives (FP), and False Negatives (FN) for each class in the confusion matrix.
8. Plot the confusion matrix using Plotly, showing the distribution of predicted vs. true labels.
9. Create a visualization using Plotly to display True Positives, False Positives, and False Negatives per class, providing insights into model performance on different emotion categories.
10. Loads a pre-trained <ResNet> model for emotion recognition from a specified file.
11. Defines the path to a folder containing images for emotion recognition.
12. Initializes the MTCNN (Multi-Task Cascaded Convolutional Neural Network) for face detection.
13. Specifies a list of emotion labels - ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'].
14. Iterates over files in the specified image folder.
15. For each image file, it loads the image and performs face detection using MTCNN.
16. Predicts emotions for detected faces in the image using the loaded <ResNet> model.
17. Keeps track of the count of each predicted emotion for the image.
18. Displays the image with detected faces and predicted emotions, along with bounding boxes and text labels indicating the predicted emotions.
19. Prints the counts of each predicted emotion for the image by adding labele like ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'] for all images.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Instaling necessary libraries

!pip install mtcnn
!pip install tensorflow


In [ ]:
# RESNET

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, Add
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Residual Block
def residual_block(inputs, num_filters, strides=(1, 1), use_projection=False):
    x = Conv2D(num_filters, kernel_size=(3, 3), strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    if use_projection:
        # Use projection shortcut to match dimensions
        shortcut = Conv2D(num_filters, kernel_size=(1, 1), strides=strides, padding='valid')(inputs)
        shortcut = BatchNormalization()(shortcut)
    else:
        # Identity shortcut
        shortcut = inputs

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Define ResNet Model
def ResNet(input_shape=(48, 48, 1), num_classes=7):
    inputs = Input(shape=input_shape)

    # Initial Convolutional layer
    x = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual blocks
    x = residual_block(x, num_filters=64, use_projection=False)
    x = residual_block(x, num_filters=64, use_projection=False)
    x = residual_block(x, num_filters=64, use_projection=False)

    x = residual_block(x, num_filters=128, strides=(2, 2), use_projection=True)
    x = residual_block(x, num_filters=128, use_projection=False)
    x = residual_block(x, num_filters=128, use_projection=False)

    x = GlobalAveragePooling2D()(x)

    # Fully connected layer
    x = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs, x)
    return model

# Create ResNet model
model = ResNet()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("/content/drive/MyDrive/emo/data/emotion_resnet.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

# Plot accuracy and loss curves
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Confusion matrix, IOU, TP, FP, FN - RESNET.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_resnet.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(conf_matrix)

# Calculate IoU for each class
def calculate_iou(conf_matrix):
    ious = []
    num_classes = conf_matrix.shape[0]
    tp_list, fp_list, fn_list = [], [], []
    for c in range(num_classes):
        TP = conf_matrix[c, c]
        FP = np.sum(conf_matrix[:, c]) - TP
        FN = np.sum(conf_matrix[c, :]) - TP
        IoU = TP / (TP + FP + FN)
        ious.append(IoU)
        tp_list.append(TP)
        fp_list.append(FP)
        fn_list.append(FN)
    return ious, tp_list, fp_list, fn_list

ious, tp_list, fp_list, fn_list = calculate_iou(conf_matrix)
for i, iou in enumerate(ious):
    print(f"IoU for class {i}: {iou}")

# Define emotion labels
emotion_labels = label_encoder.classes_

# Plot confusion matrix using Plotly
fig_conf_matrix = px.imshow(conf_matrix,
                            labels=dict(x="Predicted Label", y="True Label", color="Count"),
                            x=emotion_labels,
                            y=emotion_labels,
                            title="Confusion Matrix")
fig_conf_matrix.update_layout(coloraxis_showscale=True)
fig_conf_matrix.show()

# Plot TP, FP, FN using Plotly
metrics_data = pd.DataFrame({
    "Class": emotion_labels,
    "True Positives (TP)": tp_list,
    "False Positives (FP)": fp_list,
    "False Negatives (FN)": fn_list
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["True Positives (TP)"], name='TP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Positives (FP)"], name='FP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Negatives (FN)"], name='FN'))

fig_metrics.update_layout(
    barmode='group',
    title="True Positives, False Positives, and False Negatives per Class",
    xaxis_title="Class",
    yaxis_title="Count",
    legend_title="Metrics"
)

fig_metrics.show()


In [ ]:
# Detecting Group emotions - RESNET.

import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import os


# Load the model with custom objects
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_resnet.h5", custom_objects={'SpatialAttention': SpatialAttention})

# Path to the folder containing images
image_folder = "/content/drive/MyDrive/emo/data"

# Initialize MTCNN for face detection
detector = MTCNN()

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Iterate over files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image file
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(image_folder, filename)

        # Load the input image
        image = cv2.imread(image_path)

        # Perform face detection using MTCNN
        detections = detector.detect_faces(image)

        # Initialize emotion counters
        emotion_counts = {emotion: 0 for emotion in emotion_labels}

        # Predict emotions for the detected faces
        for detection in detections:
            x, y, w, h = detection['box']
            face_roi = image[y:y+h, x:x+w]
            face = cv2.resize(face_roi, (48, 48))  # Resize to 48x48 for emotion recognition
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            face = face.astype('float32') / 255.0  # Normalize pixel values
            face = np.expand_dims(face, axis=0)  # Add batch dimension
            face = np.expand_dims(face, axis=-1)  # Add channel dimension
            prediction = model.predict(face)
            emotion_index = np.argmax(prediction)
            emotion = emotion_labels[emotion_index]
            emotion_counts[emotion] += 1
            cv2.putText(image, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Print emotion counts for the image
        print(f"Image: {filename}")
        for emotion, count in emotion_counts.items():
            print(f"Number of {emotion}: {count}")

        # Display the image with predicted emotions
        cv2_imshow(image)


In [ ]:
# Hierarchical Convolutional Block

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Multiply
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2


# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model with attention
def FER_Model(input_shape=(48, 48, 1)):
    visible = Input(shape=input_shape, name='input')
    num_classes = 7

    # The first block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2, 2), name='pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name='drop1_1')(pool1_1)

    # The second block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_3')(conv2_2)
    conv2_3 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2, 2), name='pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name='drop2_1')(pool2_1)

    # The third block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2, 2), name='pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name='drop3_1')(pool3_1)

    # The fourth block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2, 2), name='pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name='drop4_1')(pool4_1)

    # The fifth block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_4')(conv5_3)
    conv5_4 = BatchNormalization()(conv5_4)
    pool5_1 = MaxPooling2D(pool_size=(2, 2), name='pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name='drop5_1')(pool5_1)

    # Flatten and output
    flatten = Flatten(name='flatten')(drop5_1)
    output = Dense(num_classes, activation='softmax', name='output')(flatten)

    # Create model
    model = Model(inputs=visible, outputs=output)

    return model

# Create an instance of the model
model = FER_Model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("/content/drive/MyDrive/emo/data/emotion_Hierarchical.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

# Plot accuracy and loss curves
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Confusion matrix, IOU, TP, FP, FN - Hierarchical.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_Hierarchical.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(conf_matrix)

# Calculate IoU for each class
def calculate_iou(conf_matrix):
    ious = []
    num_classes = conf_matrix.shape[0]
    tp_list, fp_list, fn_list = [], [], []
    for c in range(num_classes):
        TP = conf_matrix[c, c]
        FP = np.sum(conf_matrix[:, c]) - TP
        FN = np.sum(conf_matrix[c, :]) - TP
        IoU = TP / (TP + FP + FN)
        ious.append(IoU)
        tp_list.append(TP)
        fp_list.append(FP)
        fn_list.append(FN)
    return ious, tp_list, fp_list, fn_list

ious, tp_list, fp_list, fn_list = calculate_iou(conf_matrix)
for i, iou in enumerate(ious):
    print(f"IoU for class {i}: {iou}")

# Define emotion labels
emotion_labels = label_encoder.classes_

# Plot confusion matrix using Plotly
fig_conf_matrix = px.imshow(conf_matrix,
                            labels=dict(x="Predicted Label", y="True Label", color="Count"),
                            x=emotion_labels,
                            y=emotion_labels,
                            title="Confusion Matrix")
fig_conf_matrix.update_layout(coloraxis_showscale=True)
fig_conf_matrix.show()

# Plot TP, FP, FN using Plotly
metrics_data = pd.DataFrame({
    "Class": emotion_labels,
    "True Positives (TP)": tp_list,
    "False Positives (FP)": fp_list,
    "False Negatives (FN)": fn_list
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["True Positives (TP)"], name='TP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Positives (FP)"], name='FP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Negatives (FN)"], name='FN'))

fig_metrics.update_layout(
    barmode='group',
    title="True Positives, False Positives, and False Negatives per Class",
    xaxis_title="Class",
    yaxis_title="Count",
    legend_title="Metrics"
)

fig_metrics.show()

In [ ]:
# Detecting Group emotions - Hierarchical

import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import os


# Load the model with custom objects
# model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_Hierarchical.h5", custom_objects={'SpatialAttention': SpatialAttention})
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_Hierarchical.h5")


# Path to the folder containing images
image_folder = "/content/drive/MyDrive/emo/data"

# Initialize MTCNN for face detection
detector = MTCNN()

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Iterate over files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image file
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(image_folder, filename)

        # Load the input image
        image = cv2.imread(image_path)

        # Perform face detection using MTCNN
        detections = detector.detect_faces(image)

        # Initialize emotion counters
        emotion_counts = {emotion: 0 for emotion in emotion_labels}

        # Predict emotions for the detected faces
        for detection in detections:
            x, y, w, h = detection['box']
            face_roi = image[y:y+h, x:x+w]
            face = cv2.resize(face_roi, (48, 48))  # Resize to 48x48 for emotion recognition
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            face = face.astype('float32') / 255.0  # Normalize pixel values
            face = np.expand_dims(face, axis=0)  # Add batch dimension
            face = np.expand_dims(face, axis=-1)  # Add channel dimension
            prediction = model.predict(face)
            emotion_index = np.argmax(prediction)
            emotion = emotion_labels[emotion_index]
            emotion_counts[emotion] += 1
            cv2.putText(image, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Print emotion counts for the image
        print(f"Image: {filename}")
        for emotion, count in emotion_counts.items():
            print(f"Number of {emotion}: {count}")

        # Display the image with predicted emotions
        cv2_imshow(image)

In [ ]:
# Convolution Block Attention Module (CBAM).

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Multiply, Add
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, concatenate, GlobalAveragePooling2D, Reshape
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Lambda, Concatenate
from keras import backend as K

# Define the Convolutional Block Attention Module (CBAM)
def cbam_block(cbam_feature, ratio=8):
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = Dense(channel // ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel // ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel)

    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1, 1, channel))(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel // ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel)

    cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    return Multiply()([input_feature, cbam_feature])

def spatial_attention(input_feature):
    cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters=1,
                          kernel_size=7, strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)

    return cbam_feature

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model with CBAM
def FER_Model(input_shape=(48, 48, 1)):
    visible = Input(shape=input_shape, name='input')
    num_classes = 7

    # The first block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_1 = cbam_block(conv1_1)  # Add CBAM here
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    conv1_2 = cbam_block(conv1_2)  # Add CBAM here
    pool1_1 = MaxPooling2D(pool_size=(2, 2), name='pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name='drop1_1')(pool1_1)

     # The second block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_1 = cbam_block(conv2_1)  # Add CBAM here
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_2 = cbam_block(conv2_2)  # Add CBAM here
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_3')(conv2_2)
    conv2_3 = BatchNormalization()(conv2_3)
    conv2_3 = cbam_block(conv2_3)  # Add CBAM here
    pool2_1 = MaxPooling2D(pool_size=(2, 2), name='pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name='drop2_1')(pool2_1)

    # The third block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    pool3_1 = MaxPooling2D(pool_size=(2, 2), name='pool3_1')(conv3_3)
    drop3_1 = Dropout(0.3, name='drop3_1')(pool3_1)

    # The fourth block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    pool4_1 = MaxPooling2D(pool_size=(2, 2), name='pool4_1')(conv4_3)
    drop4_1 = Dropout(0.3, name='drop4_1')(pool4_1)

    # The fifth block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    pool5_1 = MaxPooling2D(pool_size=(2, 2), name='pool5_1')(conv5_3)
    drop5_1 = Dropout(0.3, name='drop5_1')(pool5_1)

    # Global average pooling
    global_avg_pool = GlobalAveragePooling2D()(drop5_1)

    # Output layer
    output = Dense(num_classes, activation='softmax', name='output')(global_avg_pool)

    # Create model
    model = Model(inputs=visible, outputs=output)

    return model

# Create an instance of the model
model = FER_Model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("/content/drive/MyDrive/emo/data/emotion_3_cbam.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

# Plot accuracy and loss curves
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Confusion matrix, IOU, TP, FP, FN - Convolution Block Attention Module (CBAM).

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_cbam.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(conf_matrix)

# Calculate IoU for each class
def calculate_iou(conf_matrix):
    ious = []
    num_classes = conf_matrix.shape[0]
    tp_list, fp_list, fn_list = [], [], []
    for c in range(num_classes):
        TP = conf_matrix[c, c]
        FP = np.sum(conf_matrix[:, c]) - TP
        FN = np.sum(conf_matrix[c, :]) - TP
        IoU = TP / (TP + FP + FN)
        ious.append(IoU)
        tp_list.append(TP)
        fp_list.append(FP)
        fn_list.append(FN)
    return ious, tp_list, fp_list, fn_list

ious, tp_list, fp_list, fn_list = calculate_iou(conf_matrix)
for i, iou in enumerate(ious):
    print(f"IoU for class {i}: {iou}")

# Define emotion labels
emotion_labels = label_encoder.classes_

# Plot confusion matrix using Plotly
fig_conf_matrix = px.imshow(conf_matrix,
                            labels=dict(x="Predicted Label", y="True Label", color="Count"),
                            x=emotion_labels,
                            y=emotion_labels,
                            title="Confusion Matrix")
fig_conf_matrix.update_layout(coloraxis_showscale=True)
fig_conf_matrix.show()

# Plot TP, FP, FN using Plotly
metrics_data = pd.DataFrame({
    "Class": emotion_labels,
    "True Positives (TP)": tp_list,
    "False Positives (FP)": fp_list,
    "False Negatives (FN)": fn_list
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["True Positives (TP)"], name='TP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Positives (FP)"], name='FP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Negatives (FN)"], name='FN'))

fig_metrics.update_layout(
    barmode='group',
    title="True Positives, False Positives, and False Negatives per Class",
    xaxis_title="Class",
    yaxis_title="Count",
    legend_title="Metrics"
)

fig_metrics.show()


In [ ]:
# Detecting Group emotions - Convolution Block Attention Module (CBAM).

import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import os


# Load the model with custom objects
# model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_cbam.h5", custom_objects={'SpatialAttention': SpatialAttention})
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_cbam.h5")


# Path to the folder containing images
image_folder = "/content/drive/MyDrive/emo/data"

# Initialize MTCNN for face detection
detector = MTCNN()

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Iterate over files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image file
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(image_folder, filename)

        # Load the input image
        image = cv2.imread(image_path)

        # Perform face detection using MTCNN
        detections = detector.detect_faces(image)

        # Initialize emotion counters
        emotion_counts = {emotion: 0 for emotion in emotion_labels}

        # Predict emotions for the detected faces
        for detection in detections:
            x, y, w, h = detection['box']
            face_roi = image[y:y+h, x:x+w]
            face = cv2.resize(face_roi, (48, 48))  # Resize to 48x48 for emotion recognition
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            face = face.astype('float32') / 255.0  # Normalize pixel values
            face = np.expand_dims(face, axis=0)  # Add batch dimension
            face = np.expand_dims(face, axis=-1)  # Add channel dimension
            prediction = model.predict(face)
            emotion_index = np.argmax(prediction)
            emotion = emotion_labels[emotion_index]
            emotion_counts[emotion] += 1
            cv2.putText(image, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Print emotion counts for the image
        print(f"Image: {filename}")
        for emotion, count in emotion_counts.items():
            print(f"Number of {emotion}: {count}")

        # Display the image with predicted emotions
        cv2_imshow(image)

In [ ]:
# Enhanced Spatial Attention.

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Multiply, concatenate, Activation
import matplotlib.pyplot as plt

# Define the Enhanced Spatial Attention layer
class EnhancedSpatialAttention(tf.keras.layers.Layer):
    def __init__(self, kernel_size=7):
        super(EnhancedSpatialAttention, self).__init__()
        self.kernel_size = kernel_size
        self.conv1 = Conv2D(1, kernel_size=self.kernel_size, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        attention = self.conv1(concat)
        return Multiply()([inputs, attention])

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the enhanced FER model
def Enhanced_FER_Model(input_shape=(48, 48, 1)):
    visible = Input(shape=input_shape, name='input')
    num_classes = 7

    # The first block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    attention1 = EnhancedSpatialAttention()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name='conv1_2')(attention1)
    conv1_2 = BatchNormalization()(conv1_2)
    drop1_1 = Dropout(0.3, name='drop1_1')(conv1_2)

    # The second block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name='conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    attention2 = EnhancedSpatialAttention()(conv2_2)
    drop2_1 = Dropout(0.3, name='drop2_1')(attention2)

    # The third block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', dilation_rate=2, name='conv3_3')(conv3_2)  # Dilated convolution
    conv3_3 = BatchNormalization()(conv3_3)
    attention3 = EnhancedSpatialAttention()(conv3_3)
    drop3_1 = Dropout(0.3, name='drop3_1')(attention3)

    # The fourth block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name='conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', dilation_rate=2, name='conv4_3')(conv4_2)  # Dilated convolution
    conv4_3 = BatchNormalization()(conv4_3)
    attention4 = EnhancedSpatialAttention()(conv4_3)
    drop4_1 = Dropout(0.3, name='drop4_1')(attention4)

    # The fifth block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name='conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', dilation_rate=2, name='conv5_3')(conv5_2)  # Dilated convolution
    conv5_3 = BatchNormalization()(conv5_3)
    attention5 = EnhancedSpatialAttention()(conv5_3)
    drop5_1 = Dropout(0.3, name='drop5_1')(attention5)

    # Flatten and output
    flatten = Flatten(name='flatten')(drop5_1)
    dense1 = Dense(256, activation='relu', name='dense1')(flatten)
    drop6_1 = Dropout(0.5, name='drop6_1')(dense1)
    dense2 = Dense(128, activation='relu', name='dense2')(drop6_1)
    drop6_2 = Dropout(0.5, name='drop6_2')(dense2)
    output = Dense(num_classes, activation='softmax', name='output')(drop6_2)

    # Create model
    model = Model(inputs=visible, outputs=output)

    return model

# Create an instance of the enhanced model
model = Enhanced_FER_Model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))
# history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("/content/drive/MyDrive/emo/data/emotion_spatial.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

# Plot accuracy and loss curves
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Confusion matrix, IOU, TP, FP, FN - Enhanced Spatial Attention.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_spatial.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(conf_matrix)

# Calculate IoU for each class
def calculate_iou(conf_matrix):
    ious = []
    num_classes = conf_matrix.shape[0]
    tp_list, fp_list, fn_list = [], [], []
    for c in range(num_classes):
        TP = conf_matrix[c, c]
        FP = np.sum(conf_matrix[:, c]) - TP
        FN = np.sum(conf_matrix[c, :]) - TP
        IoU = TP / (TP + FP + FN)
        ious.append(IoU)
        tp_list.append(TP)
        fp_list.append(FP)
        fn_list.append(FN)
    return ious, tp_list, fp_list, fn_list

ious, tp_list, fp_list, fn_list = calculate_iou(conf_matrix)
for i, iou in enumerate(ious):
    print(f"IoU for class {i}: {iou}")

# Define emotion labels
emotion_labels = label_encoder.classes_

# Plot confusion matrix using Plotly
fig_conf_matrix = px.imshow(conf_matrix,
                            labels=dict(x="Predicted Label", y="True Label", color="Count"),
                            x=emotion_labels,
                            y=emotion_labels,
                            title="Confusion Matrix")
fig_conf_matrix.update_layout(coloraxis_showscale=True)
fig_conf_matrix.show()

# Plot TP, FP, FN using Plotly
metrics_data = pd.DataFrame({
    "Class": emotion_labels,
    "True Positives (TP)": tp_list,
    "False Positives (FP)": fp_list,
    "False Negatives (FN)": fn_list
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["True Positives (TP)"], name='TP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Positives (FP)"], name='FP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Negatives (FN)"], name='FN'))

fig_metrics.update_layout(
    barmode='group',
    title="True Positives, False Positives, and False Negatives per Class",
    xaxis_title="Class",
    yaxis_title="Count",
    legend_title="Metrics"
)

fig_metrics.show()


In [ ]:
# Detecting Group emotions - Enhanced Spatial Attention.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

# Load the FER2013 dataset
df = pd.read_csv("/content/drive/MyDrive/emo/data/fer2013.csv")

# Extract images and labels
pixels = df["pixels"].tolist()
images = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.array(face).reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    images.append(face.astype('float32'))

X = np.array(images)
y = df["emotion"].values

# Normalize the pixel values
X /= 255.0

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=7)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_spatial.h5")

# Evaluate the model
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

# Print classification report and confusion matrix
print(classification_report(np.argmax(y_test, axis=1), y_pred))
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(conf_matrix)

# Calculate IoU for each class
def calculate_iou(conf_matrix):
    ious = []
    num_classes = conf_matrix.shape[0]
    tp_list, fp_list, fn_list = [], [], []
    for c in range(num_classes):
        TP = conf_matrix[c, c]
        FP = np.sum(conf_matrix[:, c]) - TP
        FN = np.sum(conf_matrix[c, :]) - TP
        IoU = TP / (TP + FP + FN)
        ious.append(IoU)
        tp_list.append(TP)
        fp_list.append(FP)
        fn_list.append(FN)
    return ious, tp_list, fp_list, fn_list

ious, tp_list, fp_list, fn_list = calculate_iou(conf_matrix)
for i, iou in enumerate(ious):
    print(f"IoU for class {i}: {iou}")

# Define emotion labels
emotion_labels = label_encoder.classes_

# Plot confusion matrix using Plotly
fig_conf_matrix = px.imshow(conf_matrix,
                            labels=dict(x="Predicted Label", y="True Label", color="Count"),
                            x=emotion_labels,
                            y=emotion_labels,
                            title="Confusion Matrix")
fig_conf_matrix.update_layout(coloraxis_showscale=True)
fig_conf_matrix.show()

# Plot TP, FP, FN using Plotly
metrics_data = pd.DataFrame({
    "Class": emotion_labels,
    "True Positives (TP)": tp_list,
    "False Positives (FP)": fp_list,
    "False Negatives (FN)": fn_list
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["True Positives (TP)"], name='TP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Positives (FP)"], name='FP'))
fig_metrics.add_trace(go.Bar(x=metrics_data["Class"], y=metrics_data["False Negatives (FN)"], name='FN'))

fig_metrics.update_layout(
    barmode='group',
    title="True Positives, False Positives, and False Negatives per Class",
    xaxis_title="Class",
    yaxis_title="Count",
    legend_title="Metrics"
)

fig_metrics.show()

In [ ]:
# Detecting Group emotions - Enhanced Spatial Attention.

import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow
import os


# Load the model with custom objects
# model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_spatial.h5", custom_objects={'SpatialAttention': SpatialAttention})
model = tf.keras.models.load_model("/content/drive/MyDrive/emo/data/emotion_spatial.h5")


# Path to the folder containing images
image_folder = "/content/drive/MyDrive/emo/data"

# Initialize MTCNN for face detection
detector = MTCNN()

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Iterate over files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image file
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(image_folder, filename)

        # Load the input image
        image = cv2.imread(image_path)

        # Perform face detection using MTCNN
        detections = detector.detect_faces(image)

        # Initialize emotion counters
        emotion_counts = {emotion: 0 for emotion in emotion_labels}

        # Predict emotions for the detected faces
        for detection in detections:
            x, y, w, h = detection['box']
            face_roi = image[y:y+h, x:x+w]
            face = cv2.resize(face_roi, (48, 48))  # Resize to 48x48 for emotion recognition
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            face = face.astype('float32') / 255.0  # Normalize pixel values
            face = np.expand_dims(face, axis=0)  # Add batch dimension
            face = np.expand_dims(face, axis=-1)  # Add channel dimension
            prediction = model.predict(face)
            emotion_index = np.argmax(prediction)
            emotion = emotion_labels[emotion_index]
            emotion_counts[emotion] += 1
            cv2.putText(image, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Print emotion counts for the image
        print(f"Image: {filename}")
        for emotion, count in emotion_counts.items():
            print(f"Number of {emotion}: {count}")

        # Display the image with predicted emotions
        cv2_imshow(image)

**Above model - 'Enhanced Spatial Attention (ESA)' crashed several time with the message - session crashed after using all available RAM / GPU.**